In [1]:
import altair as alt
from IPython.display import display
# from vega_datasets import data
from shapely.geometry import Point,LineString,Polygon
import geopandas as gpd
import json
import pandas as pd
import matplotlib.pyplot as plt
import random
from shapely.geometry import shape, mapping

In [2]:
complaints_path = 'CDPH_Environmental_Complaints.csv'
complaints_df = pd.read_csv(complaints_path)

# Convert all values in the "COMPLAINT TYPE" column to uppercase
complaints_df['COMPLAINT TYPE'] = complaints_df['COMPLAINT TYPE'].str.upper()

filtered_df = complaints_df[['INSPECTOR', 'COMPLAINT TYPE']]
filtered_df.rename(columns={'COMPLAINT TYPE': 'COMPLAINT_TYPE'}, inplace=True)
filtered_df['COMPLAINT_TYPE'] = filtered_df['COMPLAINT_TYPE'].str.replace(' ', '_', regex=False)
inspector_filtered_df = filtered_df[(filtered_df['INSPECTOR'] == '55') |(filtered_df['INSPECTOR'] == '9')]
#print(inspector_filtered_df)

grouped = inspector_filtered_df.groupby(['INSPECTOR', 'COMPLAINT_TYPE']).size().reset_index(name='COUNT')

print(grouped)

df_sorted = grouped.sort_values(by=['INSPECTOR', 'COUNT'], ascending=[True, False])

# Use groupby and head(5) to get the top 5 complaints per inspector
top5_complaints_per_inspector = df_sorted.groupby('INSPECTOR').head(5)

# Reset the index of the new DataFrame
top5_complaints_per_inspector.reset_index(drop=True, inplace=True)

top5_complaints_per_inspector_sorted = top5_complaints_per_inspector.sort_values(by=['INSPECTOR', 'COMPLAINT_TYPE'], ascending=[True, True])

# Reset the index of the new DataFrame
top5_complaints_per_inspector_sorted.reset_index(drop=True, inplace=True)

# Display the new DataFrame
print(top5_complaints_per_inspector_sorted)

top5_complaints_per_inspector_sorted.to_json("JSON/top5_complaints_per_inspector_sorted.json", orient='records', indent=4)
top5_complaints_per_inspector_sorted_json= json.dumps(top5_complaints_per_inspector_sorted.to_dict(orient='records'), indent=4) 

# with open("JSON/top5_complaints_per_inspector_sorted.json", "w") as f:
#     f.write(top5_complaints_per_inspector_sorted_json)

   INSPECTOR                             COMPLAINT_TYPE  COUNT
0         55                   AIR_POLLUTION_WORK_ORDER    635
1         55                        ASBESTOS_WORK_ORDER      1
2         55                 ILLEGAL_DUMPING_WORK_ORDER    205
3         55                            NOISE_COMPLAINT     16
4         55                                      OTHER     79
5         55           PERMITS_ISSUED_BY_DOE_WORK_ORDER     15
6         55                       RECYCLING_WORK_ORDER      2
7         55  SERVICE_STATIONS/STORAGE_TANKS_WORK_ORDER      6
8         55      TOXICS_HAZARDOUS_MATERIALS_WORK_ORDER    729
9          9                             ABANDONED_SITE      1
10         9                   AIR_POLLUTION_WORK_ORDER   1787
11         9                        ASBESTOS_WORK_ORDER      6
12         9                 ILLEGAL_DUMPING_WORK_ORDER    652
13         9                            NOISE_COMPLAINT    162
14         9                                      OTHER

C:\Users\Arka\AppData\Local\Temp\ipykernel_1236\1648434499.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.rename(columns={'COMPLAINT TYPE': 'COMPLAINT_TYPE'}, inplace=True)
C:\Users\Arka\AppData\Local\Temp\ipykernel_1236\1648434499.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['COMPLAINT_TYPE'] = filtered_df['COMPLAINT_TYPE'].str.replace(' ', '_', regex=False)


In [3]:
spec = {
  "$schema": "https://vega.github.io/schema/vega/v5.json",
  "description": "A radar chart example, showing multiple dimensions in a radial layout.",
  "width": 600,
  "height": 500,
  "padding": 150,
  "autosize": {"type": "none", "contains": "padding"},
  "background": "white",
  "title": {
    "text": "Complaints by Inspector",
    "fontSize": 20,
  },
  "signals": [
    {"name": "radius", "update": "width / 2"}
  ],

  "data": [
    {
      "name": "table",
      "url": "JSON/top5_complaints_per_inspector_sorted.json"
        
      # "values": top5_complaints_per_inspector_sorted_json
    },
    {
      "name": "keys",
      "source": "table",
      "transform": [
        {
          "type": "aggregate",
          "groupby": ["COMPLAINT_TYPE"]
        }
      ]
    }
  ],

  "scales": [
    {
      "name": "angular",
      "type": "point",
      "range": {"signal": "[-PI, PI]"},
      "padding": 0.5,
      "domain": {"data": "table", "field": "COMPLAINT_TYPE"}
    },
    {
      "name": "radial",
      "type": "linear",
      "range": {"signal": "[0, radius]"},
      "zero": True,
      "nice": False,
      "domain": {"data": "table", "field": "COUNT"},
      "domainMin": 0
    },
    {
      "name": "color",
      "type": "ordinal",
      "domain": {"data": "table", "field": "INSPECTOR"},
      "range": {"scheme": "category10"}
    }
  ],

  "encode": {
    "enter": {
      "x": {"signal": "radius"},
      "y": {"signal": "radius"}
    }
  },
  "legends": [
    {
      "title": "Inspectors",
      "fill": "color",
      "labelFontSize": 12,
      "orient": "left"
    }
  ],
  "marks": [
    {
      "type": "group",
      "name": "categories",
      "zindex": 1,
      "from": {
        "facet": {"data": "table", "name": "facet", "groupby": ["INSPECTOR"]}
      },
      "marks": [
        {
          "type": "line",
          "name": "category-line",
          "from": {"data": "facet"},
          "encode": {
            "enter": {
              "interpolate": {"value": "linear-closed"},
              "x": {"signal": "scale('radial', datum.COUNT) * cos(scale('angular', datum.COMPLAINT_TYPE))"},
              "y": {"signal": "scale('radial', datum.COUNT) * sin(scale('angular', datum.COMPLAINT_TYPE))"},
              "stroke": {"scale": "color", "field": "INSPECTOR"},
              "strokeWidth": {"value": 1},
              "fill": {"scale": "color", "field": "INSPECTOR"},
              "fillOpacity": {"value": 0.1}
            }
          }
        },
        {
          "type": "text",
          "name": "value-text",
          "from": {"data": "category-line"},
          "encode": {
            "enter": {
              "x": {"signal": "datum.x"},
              "y": {"signal": "datum.y"},
              "text": {"signal": "datum.datum.COUNT"},
              "align": {"value": "center"},
              "baseline": {"value": "middle"},
              "fill": {"value": "black"}
            }
          }
        }
      ]
    },
    {
      "type": "rule",
      "name": "radial-grid",
      "from": {"data": "keys"},
      "zindex": 0,
      "encode": {
        "enter": {
          "x": {"value": 0},
          "y": {"value": 0},
          "x2": {"signal": "radius * cos(scale('angular', datum.COMPLAINT_TYPE))"},
          "y2": {"signal": "radius * sin(scale('angular', datum.COMPLAINT_TYPE))"},
          "stroke": {"value": "darkgray"},
          "strokeWidth": {"value": 1}
        }
      }
    },
    {
      "type": "text",
      "name": "key-label",
      "from": {"data": "keys"},
      "zindex": 1,
      "encode": {
        "enter": {
          "x": {"signal": "(radius + 5) * cos(scale('angular', datum.COMPLAINT_TYPE))"},
          "y": {"signal": "(radius + 5) * sin(scale('angular', datum.COMPLAINT_TYPE))"},
          "text": {"field": "COMPLAINT_TYPE"},
          "align": [
            {
              "test": "abs(scale('angular', datum.key)) > PI / 2",
              "value": "right"
            },
            {
              "value": "left"
            }
          ],
          "baseline": [
            {
              "test": "scale('angular', datum.key) > 0", "value": "top"
            },
            {
              "test": "scale('angular', datum.key) == 0", "value": "middle"
            },
            {
              "value": "bottom"
            }
          ],
          "fill": {"value": "black"},
          "fontWeight": {"value": "bold"}
        }
      }
    },
    {
      "type": "line",
      "name": "outer-line",
      "from": {"data": "radial-grid"},
      "encode": {
        "enter": {
          "interpolate": {"value": "linear-closed"},
          "x": {"field": "x2"},
          "y": {"field": "y2"},
          "stroke": {"value": "black"},
          "strokeWidth": {"value": 1}
        }
      }
    }
  ]
}

display(alt.display.html_renderer(spec), raw=True)

In [4]:
complaints_path = 'CDPH_Environmental_Complaints.csv'
complaints_df = pd.read_csv(complaints_path)
filtered_df = complaints_df[['COMPLAINT ID', 'COMPLAINT DATE']]

filtered_df['COMPLAINT DATE'] = pd.to_datetime(filtered_df['COMPLAINT DATE'])
filtered_df['YEAR'] = filtered_df['COMPLAINT DATE'].dt.year
complaints_by_year = filtered_df['YEAR'].value_counts().reset_index()
complaints_by_year.columns = ['YEAR', 'COMPLAINT COUNT']

complaints_by_year = complaints_by_year.sort_values(by='YEAR')
complaints_by_year.reset_index(drop=True, inplace=True)

first_row = complaints_by_year.iloc[0]
last_row = complaints_by_year.iloc[-1]

# Create new rows with "Begin" and "End" years
begin_row = pd.Series(["Begin", first_row['COMPLAINT COUNT']], index=['YEAR', 'COMPLAINT COUNT'])
end_row = pd.Series(["End", last_row['COMPLAINT COUNT']], index=['YEAR', 'COMPLAINT COUNT'])

# Append the new rows to the DataFrame
complaints_by_year.loc[-1] = begin_row  # adding a row
complaints_by_year.index = complaints_by_year.index + 1 
complaints_by_year = complaints_by_year.sort_index()
# complaints_by_year = complaints_by_year.append(end_row, ignore_index=True)
# complaints_by_year = pd.concat([begin_row, complaints_by_year, end_row], ignore_index=True)
complaints_by_year = pd.concat([begin_row.to_frame().T, complaints_by_year, end_row.to_frame().T], ignore_index=True)

complaints_by_year['COUNT_CHANGE'] = complaints_by_year['COMPLAINT COUNT'].diff()

complaints_by_year.at[0, 'COUNT_CHANGE'] = complaints_by_year.at[0, 'COMPLAINT COUNT']
complaints_by_year.at[len(complaints_by_year) - 1, 'COUNT_CHANGE'] = complaints_by_year.at[len(complaints_by_year) - 1, 'COMPLAINT COUNT']
print(complaints_by_year)
complaints_by_year_json= json.dumps(complaints_by_year.to_dict(orient='records'), indent=4)

# with open("JSON/complaints_by_year.json", "w") as f:
#     f.write(complaints_by_year_json)

      YEAR COMPLAINT COUNT COUNT_CHANGE
0    Begin          2271.0       2271.0
1    Begin          2271.0          0.0
2   1993.0          2271.0          0.0
3   1994.0          2792.0        521.0
4   1995.0          2143.0       -649.0
5   1996.0          2471.0        328.0
6   1997.0          2038.0       -433.0
7   1998.0          2561.0        523.0
8   1999.0          2592.0         31.0
9   2000.0          2869.0        277.0
10  2001.0          2676.0       -193.0
11  2002.0          2600.0        -76.0
12  2003.0          2547.0        -53.0
13  2004.0          2181.0       -366.0
14  2005.0          2245.0         64.0
15  2006.0          1864.0       -381.0
16  2007.0          1695.0       -169.0
17  2008.0          1386.0       -309.0
18  2009.0          1495.0        109.0
19  2010.0          1262.0       -233.0
20  2011.0          1475.0        213.0
21  2012.0          1074.0       -401.0
22  2013.0          1220.0        146.0
23  2014.0          1110.0       -110.0


C:\Users\Arka\AppData\Local\Temp\ipykernel_1236\2037134190.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['COMPLAINT DATE'] = pd.to_datetime(filtered_df['COMPLAINT DATE'])
C:\Users\Arka\AppData\Local\Temp\ipykernel_1236\2037134190.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['YEAR'] = filtered_df['COMPLAINT DATE'].dt.year


In [5]:
spec = {
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "data": {
    # "values": complaints_by_year_json
    "url": "JSON/complaints_by_year.json"
  },
  "width": 1000,
  "height": 450,
  "title": "Waterfall chart for complaints across years",
  "transform": [
    {"window": [{"op": "sum", "field": "COUNT_CHANGE", "as": "sum"}]},
    {"window": [{"op": "lead", "field": "YEAR", "as": "lead"}]},
    {
      "calculate": "datum.lead === null ? datum.YEAR : datum.lead",
      "as": "lead"
    },
    {
      "calculate": "datum.YEAR === 'End' ? 0 : datum.sum - datum.COUNT_CHANGE",
      "as": "previous_sum"
    },
    {
      "calculate": "datum.YEAR === 'End' ? datum.sum : datum.COUNT_CHANGE",
      "as": "COUNT_CHANGE"
    },
    {
      "calculate": "(datum.YEAR !== 'Begin' && datum.YEAR !== 'End' && datum.COUNT_CHANGE > 0 ? '+' : '') + datum.COUNT_CHANGE",
      "as": "text_COUNT_CHANGE"
    },
    {"calculate": "(datum.sum + datum.previous_sum) / 2", "as": "center"},
    {
      "calculate": "datum.sum < datum.previous_sum ? datum.sum : ''",
      "as": "sum_dec"
    },
    {
      "calculate": "datum.sum > datum.previous_sum ? datum.sum : ''",
      "as": "sum_inc"
    }
  ],
  "encoding": {
    "x": {
      "field": "YEAR",
      "type": "ordinal",
      "sort": True,
      "axis": {"labelAngle": 0, "title": "Months"}
    }
  },
  "layer": [
    {
      "mark": {"type": "bar", "size": 35},
      "encoding": {
        "y": {
          "field": "previous_sum",
          "type": "quantitative",
          "title": "COUNT_CHANGE"
        },
        "y2": {"field": "sum"},
        "color": {
          "condition": [
            {
              "test": "datum.YEAR === 'Begin' || datum.YEAR === 'End'",
              "value": "#f7e0b6"
            },
            {"test": "datum.sum < datum.previous_sum", "value": "#f78a64"}
          ],
          "value": "#93c4aa"
        }
      }
    },
    {
      "mark": {
        "type": "rule",
        "color": "#404040",
        "opacity": 1,
        "strokeWidth": 2,
        "xOffset": -22.5,
        "x2Offset": 22.5
      },
      "encoding": {
        "x2": {"field": "lead"},
        "y": {"field": "sum", "type": "quantitative"}
      }
    },
    {
      "mark": {"type": "text", "dy": -4, "baseline": "bottom"},
      "encoding": {
        "y": {"field": "sum_inc", "type": "quantitative"},
        "text": {"field": "sum_inc", "type": "nominal"}
      }
    },
    {
      "mark": {"type": "text", "dy": 4, "baseline": "top"},
      "encoding": {
        "y": {"field": "sum_dec", "type": "quantitative"},
        "text": {"field": "sum_dec", "type": "nominal"}
      }
    },
    {
      "mark": {"type": "text", "fontWeight": "bold", "baseline": "middle"},
      "encoding": {
        "y": {"field": "center", "type": "quantitative"},
        "text": {"field": "text_COUNT_CHANGE", "type": "nominal"},
        "color": {
          "condition": [
            {
              "test": "datum.YEAR === 'Begin' || datum.YEAR === 'End'",
              "value": "#725a30"
            }
          ],
          "value": "white"
        }
      }
    }
  ],
  "config": {"text": {"fontWeight": "bold", "color": "#404040"}}
}
display(alt.display.html_renderer(spec), raw=True)

In [6]:
complaints_path = 'CDPH_Environmental_Complaints.csv'
complaints_df = pd.read_csv(complaints_path)

# Convert all values in the "COMPLAINT TYPE" column to uppercase
complaints_df['COMPLAINT TYPE'] = complaints_df['COMPLAINT TYPE'].str.upper()

filtered_df = complaints_df[['COMPLAINT ID', 'DATA SOURCE', 'COMPLAINT DATE']]
filtered_df_sorted = filtered_df.sort_values(by=['DATA SOURCE', 'COMPLAINT DATE']).dropna(subset=['COMPLAINT DATE'])
filtered_df_sorted['COMPLAINT DATE'] = pd.to_datetime(filtered_df_sorted['COMPLAINT DATE'])

# Extract the year from the 'COMPLAINT DATE' and add it as a new column
filtered_df_sorted['YEAR'] = filtered_df_sorted['COMPLAINT DATE'].dt.year

complaints_by_year = filtered_df_sorted.groupby(['YEAR', 'DATA SOURCE']).size().reset_index(name='NUMBER OF COMPLAINTS').sort_values(by=['DATA SOURCE', 'YEAR'], ascending=[True, True])

# Group the data by 'DATA SOURCE' and count the number of entries for each source
data_source_counts = complaints_by_year['DATA SOURCE'].value_counts()

# Find the minimum count of entries between the two data sources
min_count = data_source_counts.min()

# Filter the DataFrame to keep only the first 'min_count' entries for each data source
filtered_complaints = complaints_by_year.groupby('DATA SOURCE').head(min_count)
filtered_complaints.reset_index(drop=True, inplace=True)

filtered_complaints['YEAR_ORDER'] = filtered_complaints.groupby('DATA SOURCE').cumcount() + 1

filtered_complaints_json= json.dumps(filtered_complaints.to_dict(orient='records'), indent=4)
print(filtered_complaints)

# with open("JSON/filtered_complaints.json", "w") as f:
#     f.write(filtered_complaints_json)

    YEAR                    DATA SOURCE  NUMBER OF COMPLAINTS  YEAR_ORDER
0   2011         DEPT. OF PUBLIC HEALTH                   659           1
1   2012         DEPT. OF PUBLIC HEALTH                  1074           2
2   2013         DEPT. OF PUBLIC HEALTH                  1220           3
3   2014         DEPT. OF PUBLIC HEALTH                  1110           4
4   2015         DEPT. OF PUBLIC HEALTH                   840           5
5   2016         DEPT. OF PUBLIC HEALTH                   974           6
6   2017         DEPT. OF PUBLIC HEALTH                  1009           7
7   2018         DEPT. OF PUBLIC HEALTH                   945           8
8   2019         DEPT. OF PUBLIC HEALTH                  1362           9
9   2020         DEPT. OF PUBLIC HEALTH                  1797          10
10  2021         DEPT. OF PUBLIC HEALTH                  2397          11
11  2022         DEPT. OF PUBLIC HEALTH                  2043          12
12  2023         DEPT. OF PUBLIC HEALT

C:\Users\Arka\AppData\Local\Temp\ipykernel_1236\779826335.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_complaints['YEAR_ORDER'] = filtered_complaints.groupby('DATA SOURCE').cumcount() + 1


In [7]:
spec = {
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "description": "A population pyramid for the US in 2000.",
   "data": {
    # "values": filtered_complaints_json
    "url": "JSON/filtered_complaints.json"
  },
  "spacing": 0,
  "hconcat": [{
    "transform": [{
      "filter": {"field": "DATA SOURCE", "equal": "DEPT. OF PUBLIC HEALTH"}
    }],
    "title": "DEPT. OF PUBLIC HEALTH",
    "mark": "bar",
    "encoding": {
      "y": {
        "field": "YEAR_ORDER", "sort": "descending"
      },
      "x": {
        "aggregate": "sum",
        "title": "Number of complaints",
        "field": "NUMBER OF COMPLAINTS",
        "sort": "descending"
      },
      "color": {
        "field": "DATA SOURCE",
        "scale": {"range": ["#675193", "#ca8861"]},
      }
    }
  }, {
    "width": 20,
    "mark": {
      "type": "text",
      "align": "center"
    },
    "encoding": {
      "y": {"field": "YEAR_ORDER", "type": "ordinal","axis": False, "sort": "descending"},
      "text": {"field": "YEAR_ORDER", "type": "quantitative"}
    }
  }, {
    "transform": [{
      "filter": {"field": "DATA SOURCE", "equal": "HISTORIC DEPT. OF ENVIRONMENT"}
    }],
    "title": "HISTORIC DEPT. OF ENVIRONMENT",
    "mark": "bar",
    "encoding": {
      "y": {
        "field": "NUMBER OF COMPLAINTS", 
          "axis": False,
         "sort": "descending"
      },
      "x": {
        "aggregate": "sum",
        "title": "Number of complaints",
        "field": "NUMBER OF COMPLAINTS"
      },
      "color": {
        "field": "DATA SOURCE",
        
      }
    }
  }]
}
display(alt.display.html_renderer(spec), raw=True)

In [8]:
complaints_path = 'CDPH_Environmental_Complaints.csv'
complaints_df = pd.read_csv(complaints_path)
#print(complaints_df)

filtered_df = complaints_df[["COMPLAINT TYPE", "COMPLAINT ID", "STREET TYPE"]]

street_type_complaint_counts = filtered_df.groupby("STREET TYPE")["COMPLAINT TYPE"].count().reset_index()
street_type_complaint_counts.columns = ["STREET TYPE", "NUMBER OF COMPLAINTS"]

top_5_street_types = street_type_complaint_counts.sort_values(by="NUMBER OF COMPLAINTS", ascending=False).head(5)

print(top_5_street_types)

   STREET TYPE  NUMBER OF COMPLAINTS
0          AVE                 30229
10          ST                 20371
9           RD                  2322
7           PL                  1306
3           DR                  1016


In [9]:
complaints_path = 'CDPH_Environmental_Complaints.csv'
complaints_df = pd.read_csv(complaints_path)
#print(complaints_df)

filtered_df = complaints_df[["COMPLAINT TYPE", "COMPLAINT ID", "STREET TYPE"]]

street_type_complaint_counts = filtered_df.groupby("STREET TYPE")["COMPLAINT TYPE"].count().reset_index()
street_type_complaint_counts.columns = ["STREET TYPE", "NUMBER OF COMPLAINTS"]

street_types = street_type_complaint_counts.sort_values(by="NUMBER OF COMPLAINTS", ascending=False).head(5)
print(street_types)
street_types_json= json.dumps(street_types.to_dict(orient='records'), indent=4)

# with open("JSON/street_types.json", "w") as f:
#     f.write(street_types_json)

spec = {
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "title": "Complaints by street type",

  "layer": [
    {
      "data": {
        # "values": street_types_json
        "url": "JSON/street_types.json"
      },   
      "mark": {
        "type": "arc",
        "innerRadius": 0, 
        "outerRadius": 60
      },
      "encoding": {
        "theta": {"field": "NUMBER OF COMPLAINTS", "type": "quantitative"},
        "color": {"field": "STREET TYPE", "type": "nominal"}
      }
    }
  ],
  "width": 200,
  "height": 200
}

display(alt.display.html_renderer(spec), raw=True)

   STREET TYPE  NUMBER OF COMPLAINTS
0          AVE                 30229
10          ST                 20371
9           RD                  2322
7           PL                  1306
3           DR                  1016


In [10]:
complaints_path = 'CDPH_Environmental_Complaints.csv'
complaints_df = pd.read_csv(complaints_path)

# Create a new DataFrame with only the "COMPLAINT ID" and "COMPLAINT DATE" columns
filtered_data = complaints_df[["COMPLAINT ID", "COMPLAINT DATE"]]

# Optionally, you can convert the "COMPLAINT DATE" column to a datetime data type
filtered_data["COMPLAINT DATE"] = pd.to_datetime(filtered_data["COMPLAINT DATE"])

filtered_data["COMPLAINT DATE"] = pd.to_datetime(filtered_data["COMPLAINT DATE"])

# Group the data by month and count the number of complaints in each month
monthly_complaints = filtered_data.groupby(filtered_data["COMPLAINT DATE"].dt.strftime('%Y-%m'))["COMPLAINT ID"].count().reset_index()

# Rename the columns for clarity
monthly_complaints.columns = ["Month", "Number of Complaints"]


monthly_complaints[["Year", "Month"]] = monthly_complaints["Month"].str.split('-', expand=True)

# Group the data by year and month, and count the number of complaints in each period
yearly_monthly_counts = monthly_complaints.groupby(["Year", "Month"])["Number of Complaints"].sum().reset_index()

# Rename the columns for clarity
yearly_monthly_counts.columns = ["Year", "Month", "Count"]


yearly_monthly_counts_json= json.dumps(yearly_monthly_counts.to_dict(orient='records'), indent=4)
print(yearly_monthly_counts)

# with open("JSON/yearly_monthly_counts.json", "w") as f:
#     f.write(yearly_monthly_counts_json)

C:\Users\Arka\AppData\Local\Temp\ipykernel_1236\3988001718.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["COMPLAINT DATE"] = pd.to_datetime(filtered_data["COMPLAINT DATE"])
C:\Users\Arka\AppData\Local\Temp\ipykernel_1236\3988001718.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["COMPLAINT DATE"] = pd.to_datetime(filtered_data["COMPLAINT DATE"])


     Year Month  Count
0    1993    01    129
1    1993    02    130
2    1993    03    189
3    1993    04    189
4    1993    05    173
..    ...   ...    ...
365  2023    06    223
366  2023    07    162
367  2023    08    261
368  2023    09    182
369  2023    10     39

[370 rows x 3 columns]


In [11]:
spec = {
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "data": {
    # "values": yearly_monthly_counts_json,
    "url": "JSON/yearly_monthly_counts.json"
  },
  "title": "Heatmap of count of complaints across months and years",
  "mark": "rect",
  "encoding": {
    "x": {
      "field": "Year",
      "type": "ordinal",
      "title": "Year"
    },
    "y": {
      "field": "Month",
      "type": "ordinal",
      "title": "Month"
    },
    "color": {
      "field": "Count",
      "type": "quantitative",
      "scale": {"scheme": "viridis"} 
    }
  }
}


display(alt.display.html_renderer(spec), raw=True)

In [12]:
import pandas as pd
import altair as alt

complaints_path = 'CDPH_Environmental_Complaints.csv'
complaints_df = pd.read_csv(complaints_path)

# Convert all values in the "COMPLAINT TYPE" column to uppercase
complaints_df['COMPLAINT TYPE'] = complaints_df['COMPLAINT TYPE'].str.upper()

print(complaints_df.columns)

print(complaints_df.head(7))

complaints_df_dict = complaints_df.to_dict(orient="records")
complaints_df.to_json("JSON/complaints_df.json", orient="records", indent=4)

# complaint_df_json_str = json.dumps(complaints_df_dict, indent=4)

# with open("JSON/complaints_df.json", "w") as f:
#     f.write(complaint_df_json_str)

Index(['COMPLAINT ID', 'COMPLAINT TYPE', 'ADDRESS', 'STREET NUMBER FROM',
       'STREET NUMBER TO', 'DIRECTION', 'STREET NAME', 'STREET TYPE',
       'INSPECTOR', 'COMPLAINT DATE', 'COMPLAINT DETAIL', 'INSPECTION LOG',
       'DATA SOURCE', 'Modified Date', 'LATITUDE', 'LONGITUDE', 'LOCATION'],
      dtype='object')
  COMPLAINT ID                             COMPLAINT TYPE  \
0    DOECOMP43                            NOISE COMPLAINT   
1   DOECOMP186                   AIR POLLUTION WORK ORDER   
2   DOECOMP192                            NOISE COMPLAINT   
3   DOECOMP229                   AIR POLLUTION WORK ORDER   
4   DOECOMP321                   AIR POLLUTION WORK ORDER   
5   DOECOMP334  SERVICE STATIONS/STORAGE TANKS WORK ORDER   
6   DOECOMP348                   AIR POLLUTION WORK ORDER   

                ADDRESS  STREET NUMBER FROM  STREET NUMBER TO DIRECTION  \
0          1 N STATE ST                   1               NaN         N   
1       100 N LOOMIS ST                 10

In [13]:
# Define the Vega-Lite specification for the stacked area chart
spec = {
    "data": {
        # "values": complaints_df_dict
        "url": "JSON/complaints_df.json"
    },
    "mark": "area",
    "encoding": {
        "x": {
            "field": "COMPLAINT DATE",
            "type": "temporal",
            "timeUnit": "year",
            "title": "Year"
        },
        "y": {
            "aggregate": "count",
            "type": "quantitative",
            "title": "Number of Complaints"
        },
        "color": {
            "field": "COMPLAINT TYPE",
            "type": "nominal",
            "title": "Complaint Type"
        }
    },
    "width": 600,
    "height": 400,
    "title": "Stacked Area Chart of Complaints Over the Years"
}

display(alt.display.html_renderer(spec), raw=True)

In [14]:
# Group the DataFrame by the "INSPECTOR" column and calculate the number of complaints for each inspector
inspector_complaints = complaints_df.groupby('INSPECTOR')['COMPLAINT ID'].count().reset_index()

# Rename the columns for clarity
inspector_complaints.columns = ['INSPECTOR', 'COMPLAINT COUNT']

# Sort the data by the number of complaints in ascending order
inspector_complaints = inspector_complaints.sort_values(by="COMPLAINT COUNT", ascending=False)

# Select the bottom 70 inspectors
inspector_complaints = inspector_complaints.tail(70)

inspector_complaints_dict = inspector_complaints.to_dict(orient="records")

inspector_complaints_json_str = json.dumps(inspector_complaints_dict, indent=4)

with open("JSON/inspector_complaints.json", "w") as f:
    f.write(inspector_complaints_json_str)


# Create a Vega-Lite specification for the histogram
spec = {
    "data": {
        # "values": inspector_complaints_dict
        "url": "JSON/inspector_complaints.json"
    },
    "mark": "bar",
    "encoding": {
        "x": {
            "field": "INSPECTOR",
            "type": "nominal",
            "title": "Inspector ID",
        },
        "y": {
            "field": "COMPLAINT COUNT",
            "type": "quantitative",
            "title": "Number of Complaints"
        }
    },
    "width": 600,
    "height": 400,
    "title": "Histogram of Number of Complaints by Inspector"
}

display(alt.display.html_renderer(spec), raw=True)